In [1]:
import numpy as np
import os
import pandas as pd
from scipy.stats import sem

In [2]:
def get_all_valid_files(case,date,pop):
    direct = '../../Downloads/cases_from_cluster/{}'.format(case)
    files = [f for f in os.listdir('{}/'.format(direct)) if pop in f and 'part' in f and 'no_tuning' not in f]
    #good_two = count_done('../../Downloads/cases_from_cluster/{}_two'.format(case),ptwo)
    goods ={}
    for ff in files:
        with open('{}/{}'.format(direct,ff),'r') as f:
            l = f.readlines()
        if  date in  l[-1]:
            #goods.append(f)
        
            goods[ff]=l
    return goods

In [14]:
f = get_all_valid_files('case_two', '2015-10-19 19:00:00','_32_')

In [94]:
f['updates_case_three_participant_32_7_559.txt'][11].split(' ')[14]

'1\n'

In [3]:
def get_output(lines):
    to_return = {}
    gids = {}
    for l in lines:
        try:
            p_id = l.split(' ')[11]
            steps = float(l.split(' ')[13])
            gid =  l.split(' ')[14]
            if p_id not in to_return:
                to_return[p_id]=[]
            to_return[p_id].append(steps)
            if p_id not in gids:
                gids[p_id]=gid.strip('\n')
        except Exception as e:
            print(e)
            print(l)
    return to_return,gids

In [4]:
def get_all_steps_across_files(case,date,pop):
    files = get_all_valid_files(case,date,pop)
    all_steps = []
    print(len(files))
    for f,v in files.items():
        print(f)
        od = get_output(v)
        for k,v in od[0].items():
            all_steps.extend(v)
    return all_steps

In [5]:
def get_by_date(file_lines):
    to_return = {}
    for l in file_lines:
        if 'Feb 25' in l:
            pid = l.split(' Feb 25, 2019')[1].split(' ')[1]
        
            time = pd.Timestamp(l.split(' ')[1])
            steps = float( l.split(' Feb 25, 2019')[1].split(' ')[3])
        elif 'Feb 26' in l:
            pid = l.split(' Feb 26, 2019')[1].split(' ')[1]
        
            time = pd.Timestamp(l.split(' ')[1])
            steps = float( l.split(' Feb 26, 2019')[1].split(' ')[3])
        #pid = l.split(' ')[11]
        if pid not in to_return:
            to_return[pid]={}
        to_return[pid][time]=steps
        #gid =  l.split(' ')[12]
    return to_return

In [6]:
def check_any_date(case,daylimit,pop):
    direct = '../../Downloads/cases_from_cluster/{}'.format(case)
    files = [f for f in os.listdir('{}/'.format(direct)) if pop in f and 'part' in f and 'no_tuning' not in f]
    #good_two = count_done('../../Downloads/cases_from_cluster/{}_two'.format(case),ptwo)
    all_goods ={}
    added = 0
    for ff in files:
        with open('{}/{}'.format(direct,ff),'r') as f:
            l = f.readlines()
        by_date = get_by_date(l)
        #print(by_date)
        goods = [k for k,v in by_date.items() if (sorted(v)[-1]-sorted(v)[0]).days>daylimit]
        if len(goods)==int(pop.strip('_')) and added<50:
            all_goods[ff]=by_date
            added=added+1
    return all_goods
            

In [7]:
d_two = pd.Timestamp('2015-10-19 19:00:00')

In [47]:
temp = check_any_date('case_two',41,'_32_')
len(temp)

38

In [8]:
def dict_to_steps(time_dict):
    all_steps = []
    print(len(time_dict))
    for f,d in time_dict.items():
        for k,v in d.items():
            all_steps.extend(list(v.values()))
    return np.array(all_steps)

In [21]:
'p_info 2015-10-19 10:00:00 6861 10:04PM EST on Feb 25, 2019 28 0 5.723113060403663 1'.split(' Feb 25, 2019')[1].split(' ')

['', '28', '0', '5.723113060403663', '1']

In [18]:
d_one = '2015-10-19 19:00:00'
d_two = d_one
#'2015-09-06 19:00:00'

In [164]:
print(len(steps))

61903


In [9]:
steps = dict_to_steps(check_any_date('case_two',41,'_32_'))
print(steps.mean())
print(steps.std())

0
nan
nan


In [10]:
steps = dict_to_steps(check_any_date('case_one',41,'_32_'))
print(steps.mean())
print(steps.std())

0
nan
nan


In [11]:
steps = dict_to_steps(check_any_date('case_three',41,'_32_'))
print(steps.mean())
print(steps.std())

0
nan
nan


In [170]:
steps = dict_to_steps(check_any_date('case_two',20,'_64_'))
steps.mean()

26


2.538880131984387

In [171]:
steps = dict_to_steps(check_any_date('case_one',20,'_64_'))
steps.mean()

34


2.5696280263648594

In [172]:
steps = dict_to_steps(check_any_date('case_three',20,'_64_'))
steps.mean()

10


2.5082621905963047

In [17]:
f_two = get_all_steps_across_files('case_two', d_two,'64')
print(np.array(f_two).mean())
f_three = get_all_steps_across_files('case_three',  d_two,'64')
print(np.array(f_three).mean())
f_one = get_all_steps_across_files('case_one',  d_two,'64')
print(np.array(f_one).mean())

TypeError: 'in <string>' requires string as left operand, not Timestamp

In [ ]:
def look_at_group_correlation(groupids,predictions):
    group_one = []
    group_two = []
    
    for k,v in predictions.items():
        if groupids[k]==